This is a testing notebook to expore and transform data returned by Dimensions BigQuery.

In [1]:
from datetime import datetime
import pandas as pd
from bq import BigQuery
from dimensions import DimensionsAnalytics


bq = BigQuery()
bq.add_dataset(
    name='publications',
    project_id='ucsd-discover',
    dataset='dimensions',
    table='ucsd_publications',
    billing_project_id='ucsd-discover'
)
analytics = DimensionsAnalytics(bq)

# Example filters
filters = {
    "dateRange": {
        "from": "2021-01-01",
        "to": "2022-12-30"
    },
    "citationCount": {
        "min": 1,
        "max": None
    }
}

result = await analytics.run_single_analysis(
    analysis_name='research_topics',
    dataset_name='publications',
    filters=filters
)


Executing query:

        WITH base_metrics AS (
            SELECT 
                year,
                concepts.concept,
                concepts.relevance,
                id,
                citations_count,
                metrics.field_citation_ratio,
                journal.title as journal_title,
                categories.for_2020_v2022.first_level.full[SAFE_OFFSET(0)].name as field_name,
                research_org_countries
            FROM `ucsd-discover.dimensions.ucsd_publications`,
            UNNEST(concepts) as concepts
            WHERE date_normal IS NOT NULL AND DATE(date_normal) >= DATE('2021-01-01') AND DATE(date_normal) <= DATE('2022-12-30') AND COALESCE(citations_count, 0) >= 1
            AND concepts.relevance > 0.6
        ),
        topic_summary AS (
            SELECT 
                year,
                concept,
                COUNT(*) as publication_count,
                AVG(citations_count) as avg_citations,
                AVG(field_citation_ra

In [2]:
print(result.columns)
print(len(result))

Index(['year', 'concept', 'publication_count', 'avg_citations', 'field_impact',
       'research_fields', 'collaborating_countries', 'top_journals',
       'yoy_growth', 'two_year_growth'],
      dtype='object')
197


In [3]:
print(result['concept'])

0                     T cells
1               mental health
2                older adults
3            T cell responses
4                  SARS-CoV-2
                ...          
192        mental health care
193    mental health services
194         colorectal cancer
195              focus groups
196          Kawasaki disease
Name: concept, Length: 197, dtype: object


In [4]:
def analyze_columns(df):
    """Analyze each column for potential data quality issues"""
    print("DATA QUALITY ANALYSIS")
    print("=" * 80)
    
    for column in df.columns:
        non_zero = (df[column] != 0).sum()
        total = len(df)
        non_zero_pct = (non_zero/total) * 100
        
        print(f"\nColumn: {column}")
        print(f"Non-zero values: {non_zero}/{total} ({non_zero_pct:.1f}%)")
        if df[column].dtype in ['int64', 'float64']:
            print(f"Min: {df[column].min()}")
            print(f"Max: {df[column].max()}")
            print(f"Mean: {df[column].mean():.2f}")
        print("-" * 40)

analyze_columns(result)

DATA QUALITY ANALYSIS

Column: year
Non-zero values: 197/197 (100.0%)
----------------------------------------

Column: concept
Non-zero values: 197/197 (100.0%)
----------------------------------------

Column: publication_count
Non-zero values: 197/197 (100.0%)
----------------------------------------

Column: avg_citations
Non-zero values: 197/197 (100.0%)
Min: 3.571428571428572
Max: 887.0
Mean: 44.11
----------------------------------------

Column: field_impact
Non-zero values: 197/197 (100.0%)
Min: 1.284
Max: 187.74800000000002
Mean: 15.93
----------------------------------------

Column: research_fields
Non-zero values: 197/197 (100.0%)
----------------------------------------

Column: collaborating_countries
Non-zero values: 197/197 (100.0%)
----------------------------------------

Column: top_journals
Non-zero values: 197/197 (100.0%)
----------------------------------------

Column: yoy_growth
Non-zero values: 40/197 (20.3%)
Min: -0.42857142857142855
Max: 1.5
Mean: 0.08
----